In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ml-olympiad-sustainable-urban-living/sample_submission.csv
/kaggle/input/ml-olympiad-sustainable-urban-living/train.csv
/kaggle/input/ml-olympiad-sustainable-urban-living/test.csv


In [2]:
from sklearn.preprocessing import LabelEncoder

In [3]:
x_train = pd.read_csv('/kaggle/input/ml-olympiad-sustainable-urban-living/train.csv')

In [4]:
feature_classifires_oneHot = ['Property_Type','Water_Supply']
feature_classifires_label = ['Furnishing','Power_Backup','Crime_Rate','Dust_and_Noise']
feature_numeric = ['Property_Area','Number_of_Windows','Number_of_Doors','Frequency_of_Powercuts','Traffic_Density_Score','Air_Quality_Index','Neighborhood_Review']

In [5]:
for feature in feature_classifires_oneHot:
    print(feature, x_train[feature].unique())
for feature in feature_classifires_label:
    print(feature, x_train[feature].unique())

Property_Type ['Apartment' 'Bungalow' 'Single-family home' 'Duplex' '#R%$G&867'
 'Container Home']
Water_Supply ['Once in a day - Morning' 'Once in a day - Evening' 'All time'
 'NOT MENTIONED' 'Once in two days']
Furnishing ['Semi_Furnished' 'Unfurnished' 'Fully Furnished' nan]
Power_Backup ['No' 'Yes' 'NOT MENTIONED']
Crime_Rate ['Slightly below average' 'Well below average' 'Well above average' nan
 'Slightly above average']
Dust_and_Noise ['Medium' nan 'High' 'Low']


In [6]:
x_train['Furnishing'] = x_train['Furnishing'].fillna(x_train['Furnishing'].mode()[0])
x_train['Crime_Rate'] = x_train['Crime_Rate'].fillna(x_train['Crime_Rate'].mode()[0])
x_train['Dust_and_Noise'] = x_train['Dust_and_Noise'].fillna(x_train['Dust_and_Noise'].mode()[0])

In [7]:
def imputeForNullCategorical(feature,df):
    if df[feature].isnull().any():
        df[feature].fillna(df[feature].mode()[0])
    return df

for feature in feature_classifires_oneHot+feature_classifires_label:
    x_train = imputeForNullCategorical(feature,x_train)

In [8]:
x_train['Water_Supply'] = x_train['Water_Supply'].replace('NOT MENTIONED',x_train['Water_Supply'].mode()[0])
x_train['Power_Backup'] = x_train['Power_Backup'].replace('NOT MENTIONED',x_train['Power_Backup'].mode()[0])

In [9]:
def encodeLabel(df,mapping,feature):
    values = df[feature].values
    label_encoder = LabelEncoder()
    label_encoder.fit(list(mapping.keys()))
    df_encoded_values = [mapping[value] for value in values]
    df[feature] = df_encoded_values
    return df
Crime_Rate_label_mapping = {
        'Slightly below average':1,
        'Well below average':0,
        'Well above average':3,
        'Slightly above average':2
}
Furnishing_mapping = {
    'Unfurnished' : 0,
    'Semi_Furnished' : 1,
    'Fully Furnished' : 2
}
Power_Backup_mapping = {
    'No' : 1,
    'Yes' : 2
}
Dust_and_Noise_mapping = {
    'Low' : 0,
    'Medium' : 1,
    'High' : 2
}
x_train = encodeLabel(x_train, Furnishing_mapping, 'Furnishing')
x_train = encodeLabel(x_train, Crime_Rate_label_mapping, 'Crime_Rate')
x_train = encodeLabel(x_train, Power_Backup_mapping, 'Power_Backup')
x_train = encodeLabel(x_train, Dust_and_Noise_mapping, 'Dust_and_Noise')

In [10]:
def oneHotEncode(df, feature):
    
    oneHotEncoded = pd.get_dummies(df[feature], prefix=feature)
    df = pd.concat([df, oneHotEncoded], axis=1)
    one_Hot_List = oneHotEncoded.columns.tolist()
    df.drop(feature, axis=1, inplace=True)
    return df

for feature in feature_classifires_oneHot:
     x_train = oneHotEncode(x_train, feature)

In [11]:
print(x_train.columns)

Index(['Id', 'Property_Area', 'Number_of_Windows', 'Number_of_Doors',
       'Furnishing', 'Frequency_of_Powercuts', 'Power_Backup',
       'Traffic_Density_Score', 'Crime_Rate', 'Dust_and_Noise',
       'Air_Quality_Index', 'Neighborhood_Review', 'Habitability_score',
       'Property_Type_#R%$G&867', 'Property_Type_Apartment',
       'Property_Type_Bungalow', 'Property_Type_Container Home',
       'Property_Type_Duplex', 'Property_Type_Single-family home',
       'Water_Supply_All time', 'Water_Supply_Once in a day - Evening',
       'Water_Supply_Once in a day - Morning',
       'Water_Supply_Once in two days'],
      dtype='object')


In [12]:
def imputeForNullNumerics(feature,df):
    if df[feature].isnull().any():
        df[feature].fillna(df[feature].median(),inplace=True)
    return df

for feature in feature_numeric:
    x_train = imputeForNullNumerics(feature,x_train)

/tmp/ipykernel_19/226585702.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[feature].fillna(df[feature].median(),inplace=True)
/tmp/ipykernel_19/226585702.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try u

In [13]:
x_train = x_train.drop(columns = 'Id')

In [14]:
from sklearn.preprocessing import MinMaxScaler

def normalizeFeatures(df,feature):
    scaler =MinMaxScaler()
    df[feature] = scaler.fit_transform(df[[feature]])
    return df

for feature in feature_numeric:
    normalizeFeatures(x_train, feature)

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [16]:
y_train = x_train['Habitability_score']
x_train = x_train.drop(columns=['Habitability_score'])

In [17]:
X_train, X_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [18]:
modelLinear = LinearRegression()
modelLinear.fit(X_train, y_train)

modelRF = RandomForestRegressor()
modelRF.fit(X_train, y_train)

modelSVR = SVR(kernel='rbf')
modelSVR.fit(X_train, y_train)

SVR()

In [19]:
# Predictions for Linear Regression model
Y_pred_linear = modelLinear.predict(X_test)
# Predictions for Random Forest model
Y_pred_RF = modelRF.predict(X_test)
# Predictions for Support Vector Regression model
Y_pred_SVR = modelSVR.predict(X_test)

# Calculate evaluation metrics for Linear Regression
mse_linear = mean_squared_error(y_test, Y_pred_linear)
mae_linear = mean_absolute_error(y_test, Y_pred_linear)
r2_linear = r2_score(y_test, Y_pred_linear)
print("Linear Regression Metrics:")
print("Mean Squared Error:", mse_linear)
print("Mean Absolute Error:", mae_linear)
print("R-squared (R2) Score:", r2_linear)
print()

# Calculate evaluation metrics for Random Forest
mse_RF = mean_squared_error(y_test, Y_pred_RF)
mae_RF = mean_absolute_error(y_test, Y_pred_RF)
r2_RF = r2_score(y_test, Y_pred_RF)
print("Random Forest Metrics:")
print("Mean Squared Error:", mse_RF)
print("Mean Absolute Error:", mae_RF)
print("R-squared (R2) Score:", r2_RF)
print()

# Calculate evaluation metrics for Support Vector Regression
mse_SVR = mean_squared_error(y_test, Y_pred_SVR)
mae_SVR = mean_absolute_error(y_test, Y_pred_SVR)
r2_SVR = r2_score(y_test, Y_pred_SVR)
print("Support Vector Regression Metrics:")
print("Mean Squared Error:", mse_SVR)
print("Mean Absolute Error:", mae_SVR)
print("R-squared (R2) Score:", r2_SVR)

Linear Regression Metrics:
Mean Squared Error: 82.39547207531959
Mean Absolute Error: 7.299650522870246
R-squared (R2) Score: 0.5666871216404139

Random Forest Metrics:
Mean Squared Error: 35.56253631831171
Mean Absolute Error: 4.494603370253165
R-squared (R2) Score: 0.8129787403879597

Support Vector Regression Metrics:
Mean Squared Error: 55.648898359279805
Mean Absolute Error: 5.768668479831323
R-squared (R2) Score: 0.7073457591995227


In [20]:
cat_features = ['Property_Type','Water_Supply','Furnishing','Power_Backup','Crime_Rate','Dust_and_Noise']

In [21]:
x_validation = pd.read_csv('/kaggle/input/ml-olympiad-sustainable-urban-living/test.csv')
Id = x_validation['Id']
x_validation = x_validation.drop(columns=['Id'])

In [22]:
x_validation['Furnishing'] = x_validation['Furnishing'].fillna(x_validation['Furnishing'].mode()[0])
x_validation['Crime_Rate'] = x_validation['Crime_Rate'].fillna(x_validation['Crime_Rate'].mode()[0])
x_validation['Dust_and_Noise'] = x_validation['Dust_and_Noise'].fillna(x_validation['Dust_and_Noise'].mode()[0])

In [23]:
x_validation['Water_Supply'] = x_validation['Water_Supply'].replace('NOT MENTIONED',x_validation['Water_Supply'].mode()[0])
x_validation['Power_Backup'] = x_validation['Power_Backup'].replace('NOT MENTIONED',x_validation['Power_Backup'].mode()[0])


In [24]:
x_validation['Furnishing'].unique()

array(['Unfurnished', 'Semi_Furnished', 'Fully Furnished'], dtype=object)

In [25]:

x_validation['Furnishing'].unique()

array(['Unfurnished', 'Semi_Furnished', 'Fully Furnished'], dtype=object)

In [26]:
x_validation = encodeLabel(x_validation, Furnishing_mapping, 'Furnishing')
x_validation = encodeLabel(x_validation, Crime_Rate_label_mapping, 'Crime_Rate')
x_validation = encodeLabel(x_validation, Power_Backup_mapping, 'Power_Backup')
x_validation = encodeLabel(x_validation, Dust_and_Noise_mapping, 'Dust_and_Noise')

In [27]:
for feature in feature_classifires_oneHot:
     x_validation = oneHotEncode(x_validation, feature)

In [28]:
for feature in feature_numeric:
    x_validation = imputeForNullNumerics(feature,x_validation)

/tmp/ipykernel_19/226585702.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[feature].fillna(df[feature].median(),inplace=True)
/tmp/ipykernel_19/226585702.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try u

In [29]:
for feature in feature_numeric:
    normalizeFeatures(x_validation, feature)

In [30]:
Y_valid_prediction = modelRF.predict(x_validation)